In [1]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0, heuristic=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.heuristic = heuristic

    def f(self):
        return self.path_cost + self.heuristic

    def __lt__(self, other):
        return self.f() < other.f()

    def print(self):
        if len(self.state) == 9:
            print(" ".join(self.state[0:3]))
            print(" ".join(self.state[3:6]))
            print(" ".join(self.state[6:9]))
        else:
            print(self.state)
        print("")

    def backwardList(self):
        answerList = [self]
        while answerList[0].parent != None:
            answerList.insert(0, answerList[0].parent)
        return answerList

In [2]:
import math
import copy

class Problem:
    def __init__(self, initialState, goalState):
        self.initialNode = Node(initialState)
        self.goalState = goalState
        self.size = int(math.sqrt(len(initialState)))

    def isGoal(self, node):
        if node.state == self.goalState:
            return True
        else:
            return False

    def actions(self, node):
        moves = []
        empty = node.state.index('0')
        [row, col] = [empty // self.size, empty % self.size]
        if row < self.size-1:
            moves.append((1, 0, abs(row+1 - self.goalState.index('0') // self.size) + abs(col - self.goalState.index('0') % self.size)))
        if row > 0:
            moves.append((-1, 0, abs(row-1 - self.goalState.index('0') // self.size) + abs(col - self.goalState.index('0') % self.size)))
        if col < self.size-1:
            moves.append((0, 1, abs(row - self.goalState.index('0') // self.size) + abs(col+1 - self.goalState.index('0') % self.size)))
        if col > 0:
            moves.append((0, -1, abs(row - self.goalState.index('0') // self.size) + abs(col-1 - self.goalState.index('0') % self.size)))
        return moves

    def transition(self, node, action):
        old = node.state.index('0')
        new = old + self.size*action[0] + action[1]
        transitionedNode = copy.deepcopy(node)
        lst = list(transitionedNode.state)
        lst[old], lst[new] = lst[new], lst[old]
        transitionedNode.state = ''.join(lst)
        return transitionedNode

    def expand(self, node):
        newNodes = []
        for action in self.actions(node):
            newNode = Node(self.transition(node, action).state)
            newNode.parent = node
            newNode.action = action
            newNodes.append(newNode)
        return newNodes

In [3]:
def BFS(problem):
    frontier = []
    reached = set()
    frontier.append(problem.initialNode)
    reached.add(problem.initialNode.state)
    while len(frontier) != 0:
        currentNode = frontier.pop(0)
        if problem.isGoal(currentNode):
            print("There are {0} nodes in the frontier".format(len(frontier)))
            print("There are {0} reached nodes".format(len(reached)))
            return currentNode
        for child in problem.expand(currentNode):
            if child.state not in reached:
                reached.add(child.state)
                frontier.append(child)
    return None

In [4]:
import heapq

def manhattanDistance(node, goalState, size):
    distance = 0
    for i in range(size*size):
        if node.state[i] != '0':
            [row, col] = [i // size, i % size]
            [goalRow, goalCol] = [goalState.index(node.state[i]) // size, goalState.index(node.state[i]) % size]
            distance += abs(row - goalRow) + abs(col - goalCol)
    return distance

def euclideanDistance(node, goalState, size):
    distance = 0
    for i in range(size*size):
        if node.state[i] != '0':
            [row, col] = [i // size, i % size]
            [goalRow, goalCol] = [goalState.index(node.state[i]) // size, goalState.index(node.state[i]) % size]
            distance += math.sqrt((row - goalRow)**2 + (col - goalCol)**2)
    return distance

def AStar(problem, heuristic):
    frontier = []
    reached = set()
    frontier.append((0, problem.initialNode))
    reached.add(problem.initialNode.state)
    while len(frontier) != 0:
        (_, currentNode) = heapq.heappop(frontier)
        if problem.isGoal(currentNode):
            print("There are {0} nodes in the frontier".format(len(frontier)))
            print("There are {0} reached nodes".format(len(reached)))
            return currentNode
        for child in problem.expand(currentNode):
            if child.state not in reached:
                reached.add(child.state)
                priority = child.action[2] + heuristic(child, problem.goalState, problem.size)
                heapq.heappush(frontier, (priority, child))
    return None

In [5]:
initialState = "143706582"
goalState    = "123456780"
answer = BFS(Problem(initialState, goalState))
if answer != None:
    path = answer.backwardList()
    print("These are the {0} steps to reach the goal node:".format(len(path)-1))
    for n in path:
        n.print()
else:
    print("The search has failed!")
    print("The initial state was:")
    Node(initialState).print()

There are 2626 nodes in the frontier
There are 7076 reached nodes
These are the 14 steps to reach the goal node:
1 4 3
7 0 6
5 8 2

1 0 3
7 4 6
5 8 2

1 3 0
7 4 6
5 8 2

1 3 6
7 4 0
5 8 2

1 3 6
7 4 2
5 8 0

1 3 6
7 4 2
5 0 8

1 3 6
7 4 2
0 5 8

1 3 6
0 4 2
7 5 8

1 3 6
4 0 2
7 5 8

1 3 6
4 2 0
7 5 8

1 3 0
4 2 6
7 5 8

1 0 3
4 2 6
7 5 8

1 2 3
4 0 6
7 5 8

1 2 3
4 5 6
7 0 8

1 2 3
4 5 6
7 8 0



In [6]:
initialState = "143706582"
goalState    = "123456780"
problem = Problem(initialState, goalState)
answer = AStar(problem, manhattanDistance)
if answer != None:
    path = answer.backwardList()
    print("These are the {0} steps to reach the goal node:".format(len(path)-1))
    for n in path:
        n.print()
else:
    print("The search has failed!")
    print("The initial state was:")
    Node(initialState).print()

There are 29 nodes in the frontier
There are 67 reached nodes
These are the 14 steps to reach the goal node:
1 4 3
7 0 6
5 8 2

1 0 3
7 4 6
5 8 2

1 3 0
7 4 6
5 8 2

1 3 6
7 4 0
5 8 2

1 3 6
7 4 2
5 8 0

1 3 6
7 4 2
5 0 8

1 3 6
7 4 2
0 5 8

1 3 6
0 4 2
7 5 8

1 3 6
4 0 2
7 5 8

1 3 6
4 2 0
7 5 8

1 3 0
4 2 6
7 5 8

1 0 3
4 2 6
7 5 8

1 2 3
4 0 6
7 5 8

1 2 3
4 5 6
7 0 8

1 2 3
4 5 6
7 8 0



In [7]:
initialState = "143706582"
goalState    = "123456780"
problem = Problem(initialState, goalState)
answer = AStar(problem, euclideanDistance)
if answer != None:
    path = answer.backwardList()
    print("These are the {0} steps to reach the goal node:".format(len(path)-1))
    for n in path:
        n.print()
else:
    print("The search has failed!")
    print("The initial state was:")
    Node(initialState).print()

There are 680 nodes in the frontier
There are 1784 reached nodes
These are the 60 steps to reach the goal node:
1 4 3
7 0 6
5 8 2

1 4 3
7 6 0
5 8 2

1 4 3
7 6 2
5 8 0

1 4 3
7 6 2
5 0 8

1 4 3
7 6 2
0 5 8

1 4 3
0 6 2
7 5 8

1 4 3
6 0 2
7 5 8

1 4 3
6 5 2
7 0 8

1 4 3
6 5 2
7 8 0

1 4 3
6 5 0
7 8 2

1 4 3
6 0 5
7 8 2

1 4 3
0 6 5
7 8 2

1 4 3
7 6 5
0 8 2

1 4 3
7 6 5
8 0 2

1 4 3
7 6 5
8 2 0

1 4 3
7 6 0
8 2 5

1 4 3
7 0 6
8 2 5

1 4 3
7 2 6
8 0 5

1 4 3
7 2 6
8 5 0

1 4 3
7 2 0
8 5 6

1 4 3
7 0 2
8 5 6

1 4 3
7 5 2
8 0 6

1 4 3
7 5 2
0 8 6

1 4 3
0 5 2
7 8 6

1 4 3
5 0 2
7 8 6

1 0 3
5 4 2
7 8 6

1 3 0
5 4 2
7 8 6

1 3 2
5 4 0
7 8 6

1 3 2
5 4 6
7 8 0

1 3 2
5 4 6
7 0 8

1 3 2
5 0 6
7 4 8

1 3 2
0 5 6
7 4 8

1 3 2
7 5 6
0 4 8

1 3 2
7 5 6
4 0 8

1 3 2
7 5 6
4 8 0

1 3 2
7 5 0
4 8 6

1 3 0
7 5 2
4 8 6

1 0 3
7 5 2
4 8 6

1 5 3
7 0 2
4 8 6

1 5 3
7 2 0
4 8 6

1 5 3
7 2 6
4 8 0

1 5 3
7 2 6
4 0 8

1 5 3
7 2 6
0 4 8

1 5 3
0 2 6
7 4 8

1 5 3
2 0 6
7 4 8

1 5 3
2 4 6
7 0 8

1 5 3
2 4 6
0 